MisBah Sabir 
Batch IV 
Data Science

# <center> REGRESSION using Deep Learning </center>

### <center> Assignment 1 </center>


#### Setting up Libraries

In [5]:
!pip install tensorflow
!pip install --upgrade tensorflow


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



     -------------------------------------- 284.2/284.2 MB 1.7 MB/s eta 0:00:00
     -------------------------------------- 938.7/938.7 kB 2.5 MB/s eta 0:00:00
     ---------------------------------------- 5.5/5.5 MB 2.3 MB/s eta 0:00:00
     ------------------------------------ 440.7/440.7 kB 917.7 kB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 1.2 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.15.0
    Uninstalling wrapt-1.15.0:
      Successfully uninstalled wrapt-1.15.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.13.0
    Uninstalling tensorflow-estimator-2.13.0:
      Successfully uninstalled tensorflow-estimator-2.13.0
  Attempting uninstall: keras
    Found existing installation: keras 2.13.1
    Uninstalling keras-2.13.1:
      Successfully uninstalled keras-2.13.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.13.0
  

In [6]:
#imports here
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error


ModuleNotFoundError: No module named 'tensorflow'

#### Loading Data

In [ ]:
concrete_data = pd.read_csv("https://cocl.us/concrete_data")

In [ ]:
concrete_data.head()

In [ ]:
#get data feel using pandas methods of describe/info
concrete_data.info()

In [ ]:
concrete_data.describe().T

In [ ]:
concrete_data.drop_duplicates(inplace=True)

In [ ]:
fig, axes = plt.subplots(1, 8, figsize=(30, 2))
fig.suptitle('Boxplots for outlier detection')
sns.boxplot(x=concrete_data['Cement'], ax=axes[0])
sns.boxplot(x=concrete_data['Blast Furnace Slag'], ax=axes[1])
sns.boxplot(x=concrete_data['Fly Ash'], ax=axes[2])
sns.boxplot(x=concrete_data['Water'], ax=axes[3])
sns.boxplot(x=concrete_data['Superplasticizer'], ax=axes[4])
sns.boxplot(x=concrete_data['Coarse Aggregate'], ax=axes[5])
sns.boxplot(x=concrete_data['Fine Aggregate'], ax=axes[6])
sns.boxplot(x=concrete_data['Age'], ax=axes[7])
plt.show()

In [ ]:
def outlier_imputer(columns,Inter_QR):
    '''
    Impute upper-limit values in specified columns based on their interquartile range.

    Arguments:
        column_list: A list of columns to iterate over
        iqr_factor: A number representing x in the formula:
                    Q3 + (x * IQR). Used to determine maximum threshold,
                    beyond which a point is considered an outlier.

    The IQR is computed for each column in column_list and values exceeding
    the upper threshold for each column are imputed with the upper threshold value.
    '''
    for col in columns:
        # Reassign minimum to zero
        concrete_data.loc[concrete_data[col]<0, col]

        # Calculate upper threshold
        q1 = concrete_data[col].quantile(0.25)
        q3 = concrete_data[col].quantile(0.75)
        IQR = q3 - q1
        upper_threshold = q3 + (Inter_QR * IQR)
        print(col)
        print('q3:', q3)
        print('upper_threshold:', upper_threshold)


        # Reassign values > threshold to threshold
        concrete_data.loc[concrete_data[col] > upper_threshold, col] = upper_threshold
        print(concrete_data[col].describe().T)
        print()

In [ ]:
outlier_imputer(['Blast Furnace Slag'],1.5)

In [ ]:
outlier_imputer(['Age'],1.5)

In [ ]:
outlier_imputer(['Water'],1.5)

In [ ]:
outlier_imputer(['Superplasticizer'],1.5)

In [ ]:
outlier_imputer(['Fine Aggregate'],1.5)

In [ ]:
#check for basics like shape, no of samples
concrete_data.shape

In [ ]:
#check if there are null values, and discard null values
sns.heatmap(concrete_data.isnull())

In [ ]:
concrete_data.isnull().sum()

#### Splitting data into predictors/features and target labels

In [ ]:
X = concrete_data.drop('Strength',axis=1)    # (features)

In [ ]:
y = concrete_data[['Strength']]       # targets

In [ ]:
# check if any of the features is non-numeric (categorical/ordinal). if found convert these to ohe or other accordingly
X.head()

In [ ]:
sns.heatmap(X.corr(),cmap='magma',annot=True)

### Train_Test Split

In [ ]:
#Split the data into train / test 90:10 ratio. and keep test data separate for final test
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

### Defining Model function

In [ ]:
# define the regression model, use of no. of layers / units is upto you.

model = keras.Sequential()
model.add(layers.Input(shape=(X_train.shape[1],)))
model.add(layers.Dense(80, activation='relu'))
model.add(layers.Dense(60, activation='relu'))
model.add(layers.Dense(1))

In [ ]:
model.summary()

In [ ]:
#add model.compile step

optimizer = keras.optimizers.Adam(learning_rate=1e-5)
model.compile(loss='mean_squared_error',
             optimizer=optimizer,
             metrics=['mean_absolute_error'])

### A. Building a baseline model

#### Model fitting and evaluation (with non_normalized data, epochs=50) -- repeated 50 times

In [ ]:
#use model.fit with non-normalized data.

history = model.fit(X_train,
          y_train,
          epochs=50,
          validation_split=0.2,
          batch_size=10,
          verbose=1)


In [ ]:
y_pred = model.predict(X_test)


In [ ]:
# evaluate mode using test data and print accuracy
mae = round(mean_absolute_error(y_test, y_pred),2)
mse = round(mean_squared_error(y_test, y_pred),2)

print(mae)
print(mse)

## B. Modeling and Evaluation using 'normalized data'

#### Features Normalization

In [ ]:
# use normalization of the cleaned data (obtained after null checking)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#### train_test split

In [ ]:
#train test split for Normalized Data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.10, random_state=1)

### Modeling & evaluation

In [ ]:
#create model with same no. of layers / units and see impact of normalized data on accuracy

model_b = keras.Sequential()
model_b.add(layers.Input(shape=(X_train.shape[1],)))
model_b.add(layers.Dense(80, activation='relu'))
model_b.add(layers.Dense(60, activation='relu'))
model_b.add(layers.Dense(1))



In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=1e-3)
model_b.compile(loss='mean_squared_error',
             optimizer=optimizer,
             metrics=['mean_absolute_error'])

In [ ]:
model_b.summary()

In [ ]:
history_b = model_b.fit(X_train,
          y_train,
          epochs=50,
          validation_split=0.2,
          batch_size=10,
          verbose=1)

In [ ]:
y_pred = model_b.predict(X_test)

In [ ]:
#evaluate model and print accuracy
mae_b = round(mean_absolute_error(y_test, y_pred),2)
mse_b = round(mean_squared_error(y_test, y_pred),2)

print(mae_b)
print(mse_b)

### Comparison of loss(MSE) / Accuracy between A & B (non-normalized data vs normalized data)

In [ ]:
# print mse / accuracy

print(f'MSE of Non-Normalized Data : {mse} while MSE of Normalized Data is : {mse_b}')
print(f'MAE of Non-Normalized Data : {mae} while MAE of Normalized Data is : {mae_b}')


## C. Repeating B with 100 epochs

In [ ]:
#train model at B. with 100 epochs
#X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.10, random_state=1)




In [ ]:
#create model with same no. of layers / units and see impact of normalized data on accuracy
model_c = keras.Sequential()
model_c.add(layers.Input(shape=(X_train.shape[1],)))
model_c.add(layers.Dense(80, activation='relu'))
model_c.add(layers.Dense(60, activation='relu'))
model_c.add(layers.Dense(1))


optimizer = keras.optimizers.Adam(learning_rate=1e-3)
model_c.compile(loss='mean_squared_error',
             optimizer=optimizer,
             metrics=['mean_absolute_error'])

history_c = model_c.fit(X_train,
          y_train,
          epochs=100,
          validation_split=0.2,
          batch_size=10,
          verbose=1)

In [ ]:
#evaluate and store accuracy
y_pred = model_c.predict(X_test)

mae_c = round(mean_absolute_error(y_test, y_pred),2)
mse_c = round(mean_squared_error(y_test, y_pred),2)

print(mae_c)
print(mse_c)

### Comparison of MSE between B & C (increasing epochs=50 to epochs=100)

In [ ]:
# print MSE (error) and Accuracy between B, C

# With 50 epochs
print(f'with 50 epochs, mse is {mse_b} and mae is {mae_b}')

# With 100 epochs
print(f'with 100 epochs, mse is {mse_c} and mae is {mae_c}')


## D. Optimized the number of hidden layers using counts 3,4,...

### Model  fitting and evaluation

In [ ]:
### with 3 hidden layers

# create model as above

model_d1 = keras.Sequential()
model_d1.add(layers.Input(shape=(X_train.shape[1],)))
model_d1.add(layers.Dense(80, activation='relu'))
model_d1.add(layers.Dense(60, activation='relu'))
model_d1.add(layers.Dense(20, activation='relu'))
model_d1.add(layers.Dense(1))


optimizer = keras.optimizers.Adam(learning_rate=1e-3)
model_d1.compile(loss='mean_squared_error',
             optimizer=optimizer,
             metrics=['mean_absolute_error'])

history_d1 = model_d1.fit(X_train,
          y_train,
          epochs=100,
          validation_split=0.2,
          batch_size=10,
          verbose=1)



In [ ]:
y_pred = model_d1.predict(X_test)

mae_d1 = round(mean_absolute_error(y_test, y_pred),2)
mse_d1 = round(mean_squared_error(y_test, y_pred),2)

print(mae_d1)
print(mse_d1)

In [ ]:
### with 4 hidden layers

# create model as above

model_d2 = keras.Sequential()
model_d2.add(layers.Input(shape=(X_train.shape[1],)))
model_d2.add(layers.Dense(80, activation='relu'))
model_d2.add(layers.Dense(60, activation='relu'))
model_d2.add(layers.Dense(20, activation='relu'))
model_d2.add(layers.Dense(20, activation='relu'))
model_d2.add(layers.Dense(1))


optimizer = keras.optimizers.Adam(learning_rate=1e-3)
model_d2.compile(loss='mean_squared_error',
             optimizer=optimizer,
             metrics=['mean_absolute_error'])

history_d2 = model_d2.fit(X_train,
          y_train,
          epochs=100,
          validation_split=0.2,
          batch_size=10,
          verbose=1)



In [ ]:
#repeate checking accuracy as above
y_pred = model_d2.predict(X_test)

mae_d2 = round(mean_absolute_error(y_test, y_pred),2)
mse_d2 = round(mean_squared_error(y_test, y_pred),2)

print(mae_d2)
print(mse_d2)

In [ ]:
### with 5 hidden layers

# create model as above

model_d3 = keras.Sequential()
model_d3.add(layers.Input(shape=(X_train.shape[1],)))
model_d3.add(layers.Dense(80, activation='relu'))
model_d3.add(layers.Dense(60, activation='relu'))
model_d3.add(layers.Dense(20, activation='relu'))
model_d3.add(layers.Dense(20, activation='relu'))
model_d3.add(layers.Dense(20, activation='relu'))
model_d3.add(layers.Dense(1))


optimizer = keras.optimizers.Adam(learning_rate=1e-3)
model_d3.compile(loss='mean_squared_error',
             optimizer=optimizer,
             metrics=['mean_absolute_error'])

history_d3 = model_d3.fit(X_train,
          y_train,
          epochs=100,
          validation_split=0.2,
          batch_size=10,
          verbose=1)



In [ ]:
#repeate checking accuracy as above
y_pred = model_d3.predict(X_test)

mae_d3 = round(mean_absolute_error(y_test, y_pred),2)
mse_d3 = round(mean_squared_error(y_test, y_pred),2)

print(mae_d3)
print(mse_d3)

## Comparison of MSE between C & D (increasing number of hidden layers)

In [ ]:
#print mse / accuracy

print(f'mse is {mse_c} and mae is {mae_c}')

print(f'After increasing 1 hidden layer (total 3 layers), mse is {mse_d1} and mae is {mae_d1}')
print(f'After increasing 2 hidden layer (total 4 layers), mse is {mse_d2} and mae is {mae_d2}')
print(f'After increasing 3 hidden layer (total 5 layers), mse is {mse_d3} and mae is {mae_d3}')

### Conclusion

## E. Increasing the no. of units of hidden layers (16, 32,64,96, ... style)

### Model  fitting and evaluation

***From above summary, it is evident that model with 4 hidden layers is roburst as if we move to 5 hidden layers the loss will be increasing. Hence, we will continue with using 4 hidden layers***

In [ ]:
### With neurons as 16

# create model as above

model_e_16 = keras.Sequential()
model_e_16.add(layers.Input(shape=(X_train.shape[1],)))
model_e_16.add(layers.Dense(16, activation='relu'))
model_e_16.add(layers.Dense(16, activation='relu'))
model_e_16.add(layers.Dense(16, activation='relu'))
model_e_16.add(layers.Dense(1))


optimizer = keras.optimizers.Adam(learning_rate=1e-3)
model_e_16.compile(loss='mean_squared_error',
             optimizer=optimizer,
             metrics=['mean_absolute_error'])

history_e_16 = model_e_16.fit(X_train,
          y_train,
          epochs=100,
          validation_split=0.2,
          batch_size=10,
          verbose=1)

In [ ]:
#repeat checking accuracy as above

y_pred = model_e_16.predict(X_test)

mae_e_16 = round(mean_absolute_error(y_test, y_pred),2)
mse_e_16 = round(mean_squared_error(y_test, y_pred),2)

print(mae_e_16)
print(mse_e_16)

In [ ]:
### With neurons as 32

# create model as above

model_e_32 = keras.Sequential()
model_e_32.add(layers.Input(shape=(X_train.shape[1],)))
model_e_32.add(layers.Dense(32, activation='relu'))
model_e_32.add(layers.Dense(32, activation='relu'))
model_e_32.add(layers.Dense(32, activation='relu'))
model_e_32.add(layers.Dense(1))


optimizer = keras.optimizers.Adam(learning_rate=1e-3)
model_e_32.compile(loss='mean_squared_error',
             optimizer=optimizer,
             metrics=['mean_absolute_error'])

history_e_32 = model_e_32.fit(X_train,
          y_train,
          epochs=100,
          validation_split=0.2,
          batch_size=10,
          verbose=1)

In [ ]:
#repeat checking accuracy as above

y_pred = model_e_32.predict(X_test)

mae_e_32 = round(mean_absolute_error(y_test, y_pred),2)
mse_e_32 = round(mean_squared_error(y_test, y_pred),2)

print(mae_e_32)
print(mse_e_32)

In [ ]:
### With neurons as 64

# create model as above

model_e_64 = keras.Sequential()
model_e_64.add(layers.Input(shape=(X_train.shape[1],)))
model_e_64.add(layers.Dense(64, activation='relu'))
model_e_64.add(layers.Dense(64, activation='relu'))
model_e_64.add(layers.Dense(64, activation='relu'))
model_e_64.add(layers.Dense(1))


optimizer = keras.optimizers.Adam(learning_rate=1e-3)
model_e_64.compile(loss='mean_squared_error',
             optimizer=optimizer,
             metrics=['mean_absolute_error'])

history_e_64 = model_e_64.fit(X_train,
          y_train,
          epochs=100,
          validation_split=0.2,
          batch_size=10,
          verbose=1)

In [ ]:
#repeat checking accuracy as above

y_pred = model_e_64.predict(X_test)

mae_e_64 = round(mean_absolute_error(y_test, y_pred),2)
mse_e_64 = round(mean_squared_error(y_test, y_pred),2)

print(mae_e_64)
print(mse_e_64)

In [ ]:
### With neurons as 96

# create model as above

model_e_96 = keras.Sequential()
model_e_96.add(layers.Input(shape=(X_train.shape[1],)))
model_e_96.add(layers.Dense(96, activation='relu'))
model_e_96.add(layers.Dense(96, activation='relu'))
model_e_96.add(layers.Dense(96, activation='relu'))
model_e_96.add(layers.Dense(1))


optimizer = keras.optimizers.Adam(learning_rate=1e-3)
model_e_96.compile(loss='mean_squared_error',
             optimizer=optimizer,
             metrics=['mean_absolute_error'])

history_e_96 = model_e_96.fit(X_train,
          y_train,
          epochs=100,
          validation_split=0.2,
          batch_size=10,
          verbose=1)

In [ ]:
#repeat checking accuracy as above

y_pred = model_e_96.predict(X_test)

mae_e_96 = round(mean_absolute_error(y_test, y_pred),2)
mse_e_96 = round(mean_squared_error(y_test, y_pred),2)

print(mae_e_96)
print(mse_e_96)

## Comparison of MSE between C & D (increasing number of hidden layers)

* model : initial model with 2 hidden layers, 50 epochs using non-normalized data (mae,mse) - history

* model_b : model with 2 hidden layers, 50 epochs using normalized data (mae_b,mse_b) - history_b

* model_c : model with 2 hidden layers, 100 epochs using normalized data (mae_c,mse_c) - history_c

* model_d1 : model with 3 hidden layers, 100 epochs using normalized data (mae_d1,mse_d1) - history_d1

* model_d2 : model with 4 hidden layers, 100 epochs using normalized data (mae_d2,mse_d2) - history_d2

* model_d3 : model with 5 hidden layers, 100 epochs using normalized data (mae_d3,mse_d3) - history_d3

* model_e_16 : model with 4 hidden layers, 100 epochs using normalized data, 16 neurons in each layer (mae_e_16,mse_e_16) - history_e_16

* model_e_32 : model with 4 hidden layers, 100 epochs using normalized data, 32 neurons in each layer (mae_e_32,mse_e_32) - history_e_32

* model_e_64 : model with 4 hidden layers, 100 epochs using normalized data, 64 neurons in each layer (mae_e_64,mse_e_64) - history_e_64

* model_e_96 : model with 4 hidden layers, 100 epochs using normalized data, 96 neurons in each layer (mae_e_96,mse_e_96) - history_e_96

In [ ]:
#print mse / accuracy

results = {
    'epochs':[50,50,100,100,100,100,100,100,100,100],
    'hidden_layers':[2,2,2,3,4,5,4,4,4,4],
    'neurons':['80,60','80,60','80,60','80,60,20','80,60,20,20','80,60,20,20,20','16,16,16,16','32,32,32,32','64,64,64,64','96,96,96,96'],
    'mse':[mse,mse_b,mse_c,mse_d1,mse_d2,mse_d3,mse_e_16,mse_e_32,mse_e_64,mse_e_96],
    'mae':[mae,mae_b,mae_c,mae_d1,mae_d2,mae_d3,mae_e_16,mae_e_32,mae_e_64,mae_e_96]
        }
results = pd.DataFrame(results)
results

In [ ]:
normalization_effect = {
    'type':['Normalized','Non-normalized'],
    'mse': [mse_b,mse],
    'mae': [mae_b,mae]
}

normalization_effect = pd.DataFrame(normalization_effect)
normalization_effect

In [ ]:
min_mae_record = results.loc[results['mae'].idxmin()]
min_mae_record

In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
#plots of loss / accuracy (as in chapter 4 taught) using history dictionary keys.


history_dict = history.history
loss_values = history_dict["loss"]
val_loss_values = history_dict["val_loss"]
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, "bo", label="Training loss")
plt.plot(epochs, val_loss_values, "b", label="Validation loss",color='red')
plt.title("initial model with 2 hidden layers, 50 epochs using non-normalized data")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

plt.clf()
acc = history_dict["mean_absolute_error"]
val_acc = history_dict["val_mean_absolute_error"]
plt.plot(epochs, acc, "bo", label="Training accuracy",color='magenta')
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("initial model with 2 hidden layers, 50 epochs using non-normalized data")
plt.xlabel("Epochs")
plt.ylabel("Accuracy-Mean Absolute Error")
plt.legend()
plt.show()


In [ ]:
history_dict = history_b.history
loss_values = history_dict["loss"]
val_loss_values = history_dict["val_loss"]
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, "bo", label="Training loss")
plt.plot(epochs, val_loss_values, "b", label="Validation loss",color='red')
plt.title("2 hidden layers, 50 epochs using normalized data")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

plt.clf()
acc = history_dict["mean_absolute_error"]
val_acc = history_dict["val_mean_absolute_error"]
plt.plot(epochs, acc, "bo", label="Training accuracy",color='magenta')
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("2 hidden layers, 50 epochs using normalized data")
plt.xlabel("Epochs")
plt.ylabel("Accuracy-Mean Absolute Error")
plt.legend()
plt.show()

In [ ]:
history_dict = history_c.history
loss_values = history_dict["loss"]
val_loss_values = history_dict["val_loss"]
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, "bo", label="Training loss")
plt.plot(epochs, val_loss_values, "b", label="Validation loss",color='red')
plt.title("2 hidden layers, 100 epochs using normalized data")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

plt.clf()
acc = history_dict["mean_absolute_error"]
val_acc = history_dict["val_mean_absolute_error"]
plt.plot(epochs, acc, "bo", label="Training accuracy",color='magenta')
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("2 hidden layers, 100 epochs using normalized data")
plt.xlabel("Epochs")
plt.ylabel("Accuracy-Mean Absolute Error")
plt.legend()
plt.show()

In [ ]:
history_dict = history_d1.history
loss_values = history_dict["loss"]
val_loss_values = history_dict["val_loss"]
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, "bo", label="Training loss")
plt.plot(epochs, val_loss_values, "b", label="Validation loss",color='red')
plt.title("3 hidden layers, 100 epochs")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

plt.clf()
acc = history_dict["mean_absolute_error"]
val_acc = history_dict["val_mean_absolute_error"]
plt.plot(epochs, acc, "bo", label="Training accuracy",color='magenta')
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("3 hidden layers, 100 epochs")
plt.xlabel("Epochs")
plt.ylabel("Accuracy-Mean Absolute Error")
plt.legend()
plt.show()

In [ ]:
history_dict = history_d2.history
loss_values = history_dict["loss"]
val_loss_values = history_dict["val_loss"]
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, "bo", label="Training loss")
plt.plot(epochs, val_loss_values, "b", label="Validation loss",color='red')
plt.title("4 hidden layers, 100 epochs")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

plt.clf()
acc = history_dict["mean_absolute_error"]
val_acc = history_dict["val_mean_absolute_error"]
plt.plot(epochs, acc, "bo", label="Training accuracy",color='magenta')
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("4 hidden layers, 100 epochs")
plt.xlabel("Epochs")
plt.ylabel("Accuracy-Mean Absolute Error")
plt.legend()
plt.show()

In [ ]:
history_dict = history_d3.history
loss_values = history_dict["loss"]
val_loss_values = history_dict["val_loss"]
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, "bo", label="Training loss")
plt.plot(epochs, val_loss_values, "b", label="Validation loss",color='red')
plt.title("5 hidden layers, 100 epochs")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

plt.clf()
acc = history_dict["mean_absolute_error"]
val_acc = history_dict["val_mean_absolute_error"]
plt.plot(epochs, acc, "bo", label="Training accuracy",color='magenta')
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("5 hidden layers, 100 epochs")
plt.xlabel("Epochs")
plt.ylabel("Accuracy-Mean Absolute Error")
plt.legend()
plt.show()

In [ ]:
history_dict = history_e_16.history
loss_values = history_dict["loss"]
val_loss_values = history_dict["val_loss"]
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, "bo", label="Training loss")
plt.plot(epochs, val_loss_values, "b", label="Validation loss",color='red')
plt.title("4 hidden layers, 16 neurons in each layer")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

plt.clf()
acc = history_dict["mean_absolute_error"]
val_acc = history_dict["val_mean_absolute_error"]
plt.plot(epochs, acc, "bo", label="Training accuracy",color='magenta')
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("4 hidden layers, 16 neurons in each layer")
plt.xlabel("Epochs")
plt.ylabel("Accuracy-Mean Absolute Error")
plt.legend()
plt.show()

In [ ]:
history_dict = history_e_32.history
loss_values = history_dict["loss"]
val_loss_values = history_dict["val_loss"]
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, "bo", label="Training loss")
plt.plot(epochs, val_loss_values, "b", label="Validation loss",color='red')
plt.title("4 hidden layers, 32 neurons in each layer")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

plt.clf()
acc = history_dict["mean_absolute_error"]
val_acc = history_dict["val_mean_absolute_error"]
plt.plot(epochs, acc, "bo", label="Training accuracy",color='magenta')
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("4 hidden layers, 32 neurons in each layer")
plt.xlabel("Epochs")
plt.ylabel("Accuracy-Mean Absolute Error")
plt.legend()
plt.show()

In [ ]:
history_dict = history_e_64.history
loss_values = history_dict["loss"]
val_loss_values = history_dict["val_loss"]
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, "bo", label="Training loss")
plt.plot(epochs, val_loss_values, "b", label="Validation loss",color='red')
plt.title("4 hidden layers, 64 neurons in each layer")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

plt.clf()
acc = history_dict["mean_absolute_error"]
val_acc = history_dict["val_mean_absolute_error"]
plt.plot(epochs, acc, "bo", label="Training accuracy",color='magenta')
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("4 hidden layers, 64 neurons in each layer")
plt.xlabel("Epochs")
plt.ylabel("Accuracy-Mean Absolute Error")
plt.legend()
plt.show()

In [ ]:
history_dict = history_e_96.history
loss_values = history_dict["loss"]
val_loss_values = history_dict["val_loss"]
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, "bo", label="Training loss")
plt.plot(epochs, val_loss_values, "b", label="Validation loss",color='red')
plt.title("4 hidden layers, 96 neurons in each layer")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

plt.clf()
acc = history_dict["mean_absolute_error"]
val_acc = history_dict["val_mean_absolute_error"]
plt.plot(epochs, acc, "bo", label="Training accuracy",color='magenta')
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("4 hidden layers, 96 neurons in each layer")
plt.xlabel("Epochs")
plt.ylabel("Accuracy-Mean Absolute Error")
plt.legend()
plt.show()

In [ ]:
# write what you saw when using non-normalized vs normalized data, increased epochs, layers and units.|


results

In [ ]:
normalization_effect

In [ ]:
results

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(25, 3))
fig.suptitle('Normalization of features Effect on Loss and Accuracy')
sns.barplot(data=normalization_effect, x='mae', y='type', ax=axes[0])
sns.barplot(data=normalization_effect, x='mse', y='type', ax=axes[1])
axes[0].set_ylabel('')
axes[1].set_ylabel('')


fig, axes = plt.subplots(1, 2, figsize=(25, 3))
fig.suptitle('Epochs Effect on Loss and Accuracy')
sns.lineplot(x="epochs", y="mae", data=results, label="MAE", marker="o", ax=axes[0])
sns.lineplot(x="epochs", y="mse", data=results, label="MSE", marker="o", ax=axes[1], color='green')

fig, axes = plt.subplots(1, 2, figsize=(25, 3))
fig.suptitle('Hidden Layers Effect on Loss and Accuracy')
sns.lineplot(x="hidden_layers", y="mae", data=results, label="MAE", marker="o", ax=axes[0])
sns.lineplot(x="hidden_layers", y="mse", data=results, label="MSE", marker="o", ax=axes[1], color='green')

fig, axes = plt.subplots(1, 2, figsize=(25, 3))
results['neurons_list'] = results['neurons'].apply(lambda x: [int(neuron) for neuron in x.split(',')])
results['neurons_average'] = results['neurons_list'].apply(lambda x: sum(x) / len(x))
fig.suptitle('No. of Neurons Effect on Loss and Accuracy')
sns.lineplot(x="neurons_average", y="mae", data=results[7:], label="MAE", marker="o", ax=axes[0])
sns.lineplot(x="neurons_average", y="mse", data=results[7:], label="MSE", marker="o", ax=axes[1], color='green')

plt.show()

### It is therefore evident that;
#### -normalization the data reduces MSE and MAE
#### -increasing the number of epochs reduces MSE and MAE
#### -increasing the number of layers reduces MSE and MAE
#### -and increasing the number of units also reduces MSE and MAE but to a certain level after which it starts increasing

In [ ]:
results

1. You can use model name as modelA, modelB etc. for sections A,B,....to avoid error.
2. MSE is mean squared error normally used in regression problems

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!pip install nbconvert


In [ ]:
!apt-get install texlive-xetex


In [ ]:
from nbconvert import PDFExporter
import nbformat

# Open the notebook file
notebook_file = '/content/drive/MyDrive/Assignment - 1 Regression - Week 4.ipynb'
with open(notebook_file, 'r', encoding='utf-8') as f:
    notebook_content = nbformat.read(f, as_version=4)

# Initialize PDF export
pdf_exporter = PDFExporter()
pdf_data, resources = pdf_exporter.from_notebook_node(notebook_content)

# Save the PDF
pdf_file_path = '/content/drive/MyDrive/Assignment - 1 Regression - Week 4 (Ali Affan Yaqoob).pdf'
with open(pdf_file_path, 'wb') as f:
    f.write(pdf_data)


In [ ]:
predict = model_e_64.predict(X_test)

predicted = predict[:50]

In [ ]:
actual

In [ ]:
actual = y_test.head(50)


In [ ]:
sns.scatterplot(x=actual['Strength'], y=predicted.ravel(),color='blue')
plt.ylabel('Predicted Strength')
plt.xlabel('Actual Strength')

plt.show()